# Esercitazione 12

## Esercizio 2: CNN

Ora vario l'architettura della rete, costruendone una convoluzionale. Inizio ripetendo quando fatto nell'esercizio 11.1.

In [1]:
import tensorflow as tf
from tensorflow import keras
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Flatten, Conv2D, MaxPooling2D
from keras.datasets import mnist

seed=278
np.random.seed(seed) 
tf.random.set_seed(seed)

# input image dimensions
img_rows, img_cols = 28, 28 # number of pixels 
num_classes = 10 # 10 digits

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

print('X_train shape:', X_train.shape)
print('Y_train shape:', Y_train.shape)
print()
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# reshape data, depending on Keras backend
if keras.backend.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
print('X_train shape:', X_train.shape)
print('Y_train shape:', Y_train.shape)
print()
print('X_test shape:', X_test.shape)
print('Y_test shape:', Y_test.shape)
print()
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (60000, 28, 28)
Y_train shape: (60000,)

60000 train samples
10000 test samples
X_train shape: (60000, 28, 28, 1)
Y_train shape: (60000,)

X_test shape: (10000, 28, 28, 1)
Y_test shape: (10000,)

60000 train samples
10000 test samples


Aggiungo vari strati alla rete convoluzionale: eseguo dei _pooling_, _Flatten_ e concludo con un _Dropout_ prima di ottenere il risultato finale tramite una _softmax_, che è la miglior scelta per la funzione di attivazione finale in un problema di classificazione di questo tipo.

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

def create_CNN():
    model = Sequential()
    model.add(Conv2D(10, kernel_size=(5, 5),activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(16, kernel_size=(5, 5),activation='elu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(256,input_shape=(img_rows*img_cols,), activation='relu'))
    model.add(Dense(80, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,optimizer='SGD',metrics=['accuracy'])
    return model

Ora, alleno e testo la rete

In [3]:
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols,1)

# training parameters
batch_size = 32
epochs = 10
model_CNN = create_CNN()
model_CNN.fit(X_train, Y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(X_test, Y_test))
score = model_CNN.evaluate(X_test, Y_test, verbose=1)

# print performance
print()
print('Final test loss:', score[0])
print('Final test accuracy:', score[1])

Epoch 1/10


ValueError: in user code:

    /home/emanuele/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/emanuele/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/emanuele/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/emanuele/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/emanuele/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.ylabel('model accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylabel('model loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.show()

## Esercizio 3
Ora utilizzo dei numeri da me disegnati con pimp per vedere se la predizione della mia rete allenata risulta corretta.

In [ ]:
from PIL import Image
fig = plt.figure(figsize=[14,6])
data = []

for num in range (10):
    data.append(np.array(np.zeros((28,28))))
                
for num in range (10):
    digit_filename = "./es12/num"+str(num)+".png"
    digit_in = Image.open(digit_filename).convert('L')

    ydim, xdim = digit_in.size
    pix=digit_in.load();

    for j in range(ydim):
        for i in range(xdim):
            data[num][i,j]=pix[j,i]

    data[num] /= 255

    axs = fig.add_subplot(1,10, num+1)    
    axs.imshow(data[num], cmap='gray')
    axs.axis('off')

print("Image size: "+str(xdim)+"x"+str(ydim)+" pixels")
print(data[1].shape)

In [ ]:
fig = plt.figure(figsize=[16,6])

for num in range(10):
    this_data = data[num].reshape(1,xdim*ydim)
    pred_0 = model_DNN.predict(this_data)
    plot_data = this_data.reshape(xdim,ydim)

    axs = fig.add_subplot(1,10, num+1)    
    axs.imshow(plot_data, cmap='gray')    
    axs.set_title("Digit: {}\nPredicted:    {}".format(num, np.argmax(pred_0)))    
    axs.axis('off') 
    del (this_data)